# Replicating Research from Spring

by Joshua Delgadillo

 # Update 5

### Working on:
 1. How to Jupyter Notebooks
 2. Relevant Githubs
 3. New (but similiar??) data
 4. The Alternative Algorithm

### Relevant GitHubs

Analysis using k-mer composition
https://anderson-github-classroom.github.io/csc-448-project/eagranof/

Even more analysis using k-mer composition
https://anderson-github-classroom.github.io/csc-448-project/skurdogh/ 

Vitulgin Experimentation
https://anderson-github-classroom.github.io/csc-448-project/cilg/

The Levenshtein distance experiment
https://anderson-github-classroom.github.io/csc-448-project/awengel/ 

Mutation Rate Comparison and Spike Proteins
https://anderson-github-classroom.github.io/csc-448-project/pamidi/

### New Sample Data

The [NCBI Nucleotide Database] was used to get data last time: "This database provides multiple SARS-Cov-2 sequences, but in an effort to focus our analysis, we select an arbitrarily random subset of sequences to analyze". I have a simple thought: we choose another arbitrarily random subset of sequences to analyze, which will put us at arbitrary<sup>2</sup> (aka where you want to be)

According to the [report from Spring], 677 sequences were used. Therefore, we should use 678 to literally 1-up them. We can compare our dataset to theirs to ensure everything looks right.

[NCBI Nucleotide Database]: https://www.ncbi.nlm.nih.gov/nuccore
[report from Spring]: https://nbviewer.jupyter.org/github/anderson-github-classroom/csc-448-project/blob/master/students/enewcome/supplemental-table/table.ipynb


### The Alternative Algorithm

Needleman-Wunsch Algorithm or the Smith-Waterman Algorithm can be used to compare the similiarity of two viruses. The former was used in the Spring, so we will use the Smith-Waterman Algorithm.

From what I can tell (and I'm not smart...so grains of salt), NWA fins the optimal global alignment and SWA find the best local alignment, which means: "something". I need to look back at my sequence alignment notes, but here are my current thoughts: If we run the SWA algorithm and the local alignment is enormous, then the strands of RNA are very similiar.

Some pseduo code:

~~~

Given: String s1 with length m , String s2 with length n

    // initialize matrix, M
    
    // score cells in matrix
    for i=1 to m
        for j=1 to n
        
            // initialization: max is 0
            max = 0 
            
            // first comparison: west cell (deletion)
            score = M[i][j-1] + gapScore
            if( score > max )
                max = score
            
            // second comparison: north cell (insertion)
            score = M[i-1][j] + gapScore
            if( score > max )
                max = score
            
            // last comparison: north-west cell (alignment)
            base1 = s1[j-1]
            base2 = s2[i-1]
            
            if( base1 == base2 )              // match
                alignmentScore = matchScore
            else                              // mismatch
                alignmentScore = mismatchScore
            
            score = M[i-1][j-1] + alignmentScore
            if( score > max )
                max = score
            
            // finished all comparisons
            M[i][j] = max
    
    // return completed matrix
    return M
~~~

### Stray Thoughts

1. There were doubts about Virulign, can't we just use Blast?
2. Need to verify my beliefs with bio.
3. Shoudl we use only the most recent data.
4. SWA is hopefully mostly done after our lab.

 # Update 6

### Feedback

Professor Anderson asked us to consider if we should or shouldn't implement the algorithm by hand. The primary benefit would be our familiarization with the code and control over parameters, but we give up precious time. 

We decided to try using BLAST after all, but <b>is it actually what we need?</b> Names can be very misleading.

### Proving we can use BLAST

Proof by obvious: 
~~~

lemma 1: Smith-Waterman is a "local aligment" algorithm.

B.L.A.S.T. stands for basic "local alignment" search tool.

Done.
~~~

From the [BLAST website]: "The Basic Local Alignment Search Tool (BLAST) finds regions of local similarity between sequences. The program compares nucleotide or protein sequences to sequence databases and calculates the statistical significance of matches."

BLAST uses a faster version of Smith-Waterman. A heuristic is used to eliminate sequences that are not likely to be a good local alignment. The shortcut was necessary because the algorithm is too slow when ran with large datasets. source: [The NCBI Handbook]

[BLAST website]: https://blast.ncbi.nlm.nih.gov/Blast.cgi
[The NCBI Handbook]: https://www.unmc.edu/bsbc/docs/NCBI_blast.pdf

### Sample Data

I pulled (at the time) recent data in the [NCBI] database. There are strands of SARS-CoV-1, SARS-CoV-2, and MERS-CoV. I unironically-ironically tried 678 sequences and realized I had just read about how the alogrithm struggles with large datasets. I decided to scale it back a lot becuase logic supersedes sarcasm. The function "print_fasta" does exactly what you think it does. 

[NCBI]: https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/

In [2]:
from Bio import SeqIO
import pandas as pd 
  
def print_fasta(virus_name, file_name):
    # initialize list of lists 
    data = []

    fasta_sequences = SeqIO.parse(open(file_name),'fasta')
    with open(file_name) as out_file:
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            data.append([name, sequence])

    # Create the pandas DataFrame 
    df = pd.DataFrame(data, columns = [virus_name, 'sequence']) 

    # print dataframe. 
    print(df) 
        

In [3]:
print_fasta("SARS-CoV-1", "SARS-CoV-1.fasta")
print()
print_fasta("SARS-CoV-2", "SARS-CoV-2.fasta")
print()

     SARS-CoV-1                                           sequence
0    QOQ06483.1  MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...
1    QOQ06484.1  MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...
2    QOQ06485.1  MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3    QOQ06486.1  MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWL...
4    QOQ06487.1  MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNI...
..          ...                                                ...
595  QOQ07078.1  MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPF...
596  QOQ07079.1         MIELSLIDFYLCLAFLLFLVLIMLIIFWFSLELQDHNETCHA
597  QOQ07080.1  MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWY...
598  QOQ07081.1  MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLP...
599  QOQ07082.1             MGYINVFAFPFTIYSLLLCRMNSRNYIAQVDVVNFNLT

[600 rows x 2 columns]

     SARS-CoV-2                                           sequence
0    QOP98696.1  MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...
1    QOP98697.1  MESLVPGFNEKTHVQLSLPV

### BLASTing Seqeunces

First, we need to create a blast database for our 3 datasets

In [4]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in SARS-CoV-1.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in SARS-CoV-2.fasta -dbtype prot



Building a new DB, current time: 10/26/2020 18:38:22
New DB name:   /home/jupyter-jdelga26/448/SARS-CoV-1.fasta
New DB title:  SARS-CoV-1.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/jupyter-jdelga26/448/SARS-CoV-1.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 600 sequences in 0.0198441 seconds.




Building a new DB, current time: 10/26/2020 18:38:22
New DB name:   /home/jupyter-jdelga26/448/SARS-CoV-2.fasta
New DB title:  SARS-CoV-2.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/jupyter-jdelga26/448/SARS-CoV-2.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 600 sequences in 0.0198081 seconds.




We did it! Where are our diplomas? Next, we run blast on every pair:

In [5]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query SARS-CoV-1.fasta -db SARS-CoV-2.fasta -evalue 1e-6 -num_threads 16 -out blast.txt

In [16]:
!tail -22 blast.txt


Lambda      K        H        a         alpha
   0.335    0.145    0.443    0.792     4.96 

Gapped
Lambda      K        H        a         alpha    sigma
   0.267   0.0410    0.140     1.90     42.6     43.6 

Effective search space used: 17487350


  Database: SARS-CoV-2.fasta
    Posted date:  Oct 26, 2020  6:38 PM
  Number of letters in database: 707,294
  Number of sequences in database:  600



Matrix: BLOSUM62
Gap Penalties: Existence: 11, Extension: 1
Neighboring words threshold: 11
Window for multiple hits: 40
